In [ ]:
# https://huggingface.co/docs/diffusers/using-diffusers/inference_with_lcm_lora#animatediff
# https://huggingface.co/docs/diffusers/api/pipelines/animatediff
# Must install peft

import datetime
import time
import torch
from diffusers import MotionAdapter, AnimateDiffPipeline, DDIMScheduler, LCMScheduler
from diffusers.utils import export_to_gif
from IPython.display import display

# diffusers
#pipe = AnimateDiffPipeline.from_pretrained(
#    "frankjoshua/toonyou_beta6",
#    motion_adapter=MotionAdapter.from_pretrained("guoyww/animatediff-motion-adapter-v1-5-2"),
#).to("mps")

# load SD 1.5 based finetuned model
model_id = "SG161222/Realistic_Vision_V5.1_noVAE"
#model_id = "frankjoshua/toonyou_beta6"
#model_id = "runwayml/stable-diffusion-v1-5"
pipe = AnimateDiffPipeline.from_pretrained(
    model_id, motion_adapter=MotionAdapter.from_pretrained("guoyww/animatediff-motion-adapter-v1-5-2"))
#pipe.scheduler = DDIMScheduler.from_pretrained(model_id, subfolder="scheduler", clip_sample=False, timestep_spacing="linspace", steps_offset=1)
pipe.scheduler = LCMScheduler.from_config(pipe.scheduler.config)
pipe.load_lora_weights("latent-consistency/lcm-lora-sdv1-5", adapter_name="lcm")
pipe.load_lora_weights("guoyww/animatediff-motion-lora-zoom-in", weight_name="diffusion_pytorch_model.safetensors", adapter_name="motion-lora")
pipe.set_adapters(["lcm", "motion-lora"], adapter_weights=[0.55, 1.2])
if torch.backends.mps.is_available():
    pipe.to(device="mps", dtype=torch.float16)
    print("Using MPS")
elif torch.cuda.is_available():
    torch.backends.cuda.matmul.allow_tf32 = True
    pipe.to(device="cuda", dtype=torch.float16)
    print("Using CUDA")
else:
    print("Loaded")

In [ ]:
#prompt = "best quality, masterpiece, 1girl, looking at viewer, blurry background, upper body, contemporary, dress"
#prompt = "world strongest man"
prompt = "climbing gym"
start = time.time()
generator = torch.Generator("mps").manual_seed(1) #torch.manual_seed(0)
frames = pipe(
    prompt=prompt,
    num_inference_steps=8,
    guidance_scale=1.25,
    cross_attention_kwargs={"scale": 1},
    num_frames=6,
    generator=generator
).frames[0]
print(f"Generated in {time.time()-start:.1f}s")

In [ ]:
now = str(datetime.datetime.now().replace(microsecond=0)).replace(" ", "-").replace(":", "-")
_ = export_to_gif(frames, f"out/animation-{now}.gif")